In [14]:
import torch

from datasets import LabelledPointCloudDataset
from models import LabelledPointNet, Decoder

import json
import os
from pathlib import Path

In [6]:
import numpy as np
import torch

from models import LabelledPointNet, Decoder

class LabelledPointNetAE(torch.nn.Module):
    def __init__(self, n, c):
        super(LabelledPointNetAE, self).__init__()
        self.encoder = LabelledPointNet(c)
        self.decoder = Decoder(n, c)

    def forward(self, x):
        points = x[:, :3, :]
        labels = x[:, 3:, :]
        global_feature = self.encoder(points, labels)
        # x = self.decoder(global_feature)
        return global_feature

In [7]:
device = "cuda" if torch.cuda.is_available() else "cpu"
n = 16384
c = 19

dataset = LabelledPointCloudDataset("./data/train/", c, 100, max_points=n)
loader = torch.utils.data.DataLoader(dataset, 1, shuffle=False)

model = LabelledPointNetAE(n, c)
model.load_state_dict(torch.load(
    "./runs/hpc-2025-02-22/best-model-state-dict.pt", weights_only=True, map_location=torch.device(device)
))
model.to(device)
model.eval()

LabelledPointNetAE(
  (encoder): LabelledPointNet(
    (tnet1): InputTNet(
      (conv1): Conv1d(3, 64, kernel_size=(1,), stride=(1,))
      (conv2): Conv1d(64, 128, kernel_size=(1,), stride=(1,))
      (conv3): Conv1d(128, 1024, kernel_size=(1,), stride=(1,))
      (fc1): Linear(in_features=1024, out_features=512, bias=True)
      (fc2): Linear(in_features=512, out_features=256, bias=True)
      (fc3): Linear(in_features=256, out_features=9, bias=True)
      (relu): ReLU()
      (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (bn2): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (bn3): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (bn4): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (bn5): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (tnet2): FeatureTNet(
      (conv1): Conv

In [16]:
i = 0
for batch in loader:
    x = batch
    encoding = model(x)
    with open(f"data/encodings/{i}.json", 'w') as fp:
        json.dump(encoding.squeeze().cpu().tolist(), fp)
    i += 1